In [1]:
import PySpin
import cv2
import numpy as np
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

In [6]:
def configure_camera(camera):
    # Changes amount of frame captured in camera
    width = int(2448 / 4)
    height = int(2448 / 4)

    # Reset Camera
    nodemap = camera.GetNodeMap()
    user_set_selector = PySpin.CEnumerationPtr(nodemap.GetNode("UserSetSelector"))
    default_set = user_set_selector.GetEntryByName("Default")
    user_set_selector.SetIntValue(default_set.GetValue())
    user_set_load = PySpin.CCommandPtr(nodemap.GetNode("UserSetLoad"))
    user_set_load.Execute()
    
    if camera.PixelFormat.GetAccessMode() == PySpin.RW:
        camera.PixelFormat.SetValue(PySpin.PixelFormat_RGB8)
        print("Pixel format set to color.")
    else:
        print("Color pixel format not supported.")

    # Retrieve the nodemap
    nodemap = camera.GetNodeMap()

    # Get width and height nodes
    node_width = PySpin.CIntegerPtr(nodemap.GetNode("Width"))
    node_height = PySpin.CIntegerPtr(nodemap.GetNode("Height"))
    node_width_max = node_width.GetMax()
    node_height_max = node_height.GetMax()

    # Calculate offsets to center the image
    offset_x = (node_width_max - width) // 2
    offset_y = (node_height_max - height) // 2

    # Set width and height
    node_offset_x = PySpin.CIntegerPtr(nodemap.GetNode("OffsetX"))
    node_offset_y = PySpin.CIntegerPtr(nodemap.GetNode("OffsetY"))

    node_width.SetValue(width)
    node_height.SetValue(height)
    node_offset_x.SetValue(offset_x)
    node_offset_y.SetValue(offset_y)

    camera.AcquisitionMode.SetValue(PySpin.AcquisitionMode_SingleFrame)
    camera.BeginAcquisition()

In [7]:
def capture_and_process_image(camera):
    """
    Displays live video feed from the camera.
    """
    try:
        model = YOLO("yolov8n.pt")
        relv_code = [0] #The id associated with pedestrians, used to only show peds.
        
        while True:
            image_result = camera.GetNextImage()
            if image_result.IsIncomplete():
                print('Image incomplete with image status %d ...' % image_result.GetImageStatus())
            else:
                # Convert to a format that can be displayed with OpenCV
                image_data = image_result.GetNDArray()

                #Needed if mode set to RGB8, remove if Mono8
                image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB) #CV2 needs BGR, color in RGB, just swapping positions
                
                results = model.predict(image_data, classes = relv_code, agnostic_nms = False, verbose = False)[0]

                #Array holding pixel coords and other info on detection
                #(left, top, right, bottom, class?, or confidence?) 
                # detections = np.array(results.boxes.data)
                annotator = Annotator(image_data)
                
                for r in results:
                    boxes = r.boxes
                    for box in boxes:
                        annotator.box_label(box.xyxy[0], model.names[int(box.cls)])
                        # b = box.xyxy[0] # For coordinates
                        # c = box.cls # For class
                        # annotator.box_label(b, model.names[int(c)])

                image_data = annotator.result()
                cv2.imshow('Live Camera Feed', image_data)
                
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):  # Press 'q' to quit the loop
                    break
            image_result.Release()
    finally:
        camera.EndAcquisition()

In [ ]:
def capture_and_process_image(camera, model):
    """
    Captures a single image, processes it, and displays the result.
    """
    camera.BeginAcquisition()
    image_result = camera.GetNextImage()

    # Convert to a format that can be displayed with OpenCV
    image_data = image_result.GetNDArray()
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    
    # Perform detection
    results = model.predict(image_data, classes = 0)[0] # 0 is the ped class code
    annotator = Annotator(image_data)
    for r in results:
        annotator.box_label(box.xyxy[0], model.names[int(box.cls)])

    # Display the annotated image
    annotated_image = annotator.result()
    cv2.imshow('Processed Image', annotated_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        return False

    image_result.Release()
    camera.EndAcquisition()
    return True

In [8]:
def main():
    #Halving width and height, can also be adjusted through pyspin directly, base
    width_frame = 2448 / 2 
    height = 2048 / 2
    
    # Retrieve singleton reference to system object
    system = PySpin.System.GetInstance()

    # Get current list of cameras
    cam_list = system.GetCameras()

    if cam_list.GetSize() == 0:
        print('No cameras found.')
        return False

    camera = cam_list.GetByIndex(0)
    camera.Init()

    try:
        configure_camera(camera)
        capture_and_process_image(camera)
    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
    finally:
        camera.DeInit()
        del camera
        cam_list.Clear()
        cv2.destroyAllWindows()
        system.ReleaseInstance()

In [ ]:
if __name__ == '__main__':
    main()

Pixel format set to color.
